In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import statsmodels.api as sm

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
mpl.rc('font', family=font)



In [2]:
raw=pd.read_csv('../../데이터/강수량.csv')
df=raw.copy()



In [3]:
df.head()

,일자,지역구,일일강수량,집중호우
0,2023-12-31,서초구,7.5,0
1,2023-12-31,양천구,5.5,0
2,2023-12-31,영등포구,4.5,0
3,2023-12-31,용산구,6.0,0
4,2023-12-31,은평구,4.0,0


In [7]:
# '일자'를 datetime 형식으로 변환 (혼합 형식 처리)
def parse_date(date):
    try:
        return pd.to_datetime(date, format='%Y%m%d')
    except ValueError:
        return pd.to_datetime(date)

df['일자'] = df['일자'].apply(parse_date)
df['월'] = df['일자'].dt.to_period('M')

# 월별 총 강수량 계산 (1월부터 12월까지)
monthly_rainfall_mean = df.groupby(df['일자'].dt.month)['일일강수량'].mean()

# 지역구별 월별 강수량 계산
monthly_rainfall_sum = df.groupby(['월', '지역구'])['일일강수량'].sum().reset_index()
monthly_rainfall_sum.columns = ['월', '지역구', '월별 강수량']

# 각 지역구별 연간 총 강수량 계산
annual_rainfall_sum = df.groupby(['지역구', df['일자'].dt.year])['일일강수량'].sum().reset_index()
annual_rainfall_sum.columns = ['지역구', '연도', '연간 강수량']

# 원본 데이터프레임에 연간 총 강수량 병합
df = df.merge(annual_rainfall_sum, how='left', left_on=['지역구', df['일자'].dt.year], right_on=['지역구', '연도'])

# 집중호우 여부 계산
df['집중호우'] = np.where((df['일일강수량'] >= 80) | (df['일일강수량'] >= df['연간 강수량'] * 0.1), 1, 0)

# 지역구별 집중호우 횟수 계산
heavy_rain_count = df[df['집중호우'] == 1].groupby('지역구').size()

# 연도별 총 강수량 계산
annual_rainfall_mean = df.groupby(df['일자'].dt.year)['일일강수량'].mean()

# 결과 출력
print("월별 총 강수량 평균:")
print(monthly_rainfall_mean)

print("\n지역구별 월별 총 강수량:")
print(monthly_rainfall_sum.head())

print("\n각 지역구별 연간 총 강수량:")
print(annual_rainfall_sum.head())

print("\n지역구별 집중호우 횟수:")
print(heavy_rain_count)

print("\n연도별 총 강수량 평균:")
print(annual_rainfall_mean)

월별 총 강수량 평균:
일자
1      6.867725
2      4.508861
3      9.026003
4     10.208861
5     14.021994
6     15.652158
7     20.942359
8     25.701981
9     17.120571
10    12.619190
11    14.492243
12     4.333863
Name: 일일강수량, dtype: float64

지역구별 월별 총 강수량:
         월  지역구  월별 강수량
0  2018-01  강남구     5.0
1  2018-01  강동구     4.5
2  2018-01  강북구     4.5
3  2018-01  광진구     5.5
4  2018-01  구로구     3.0

각 지역구별 연간 총 강수량:
   지역구    연도  연간 강수량
0  강남구  2018  1335.5
1  강남구  2019   939.0
2  강남구  2020  1658.5
3  강남구  2021  1092.0
4  강남구  2022  1636.0

지역구별 집중호우 횟수:
지역구
강남구     16
강동구     17
강북구     16
강서구     12
관악구     16
광진구     17
구로구     16
금천구     13
남산구      6
노원구     17
도봉구     21
동대문구    19
동작구      6
마포구     11
서대문구    13
서초구     16
성동구     14
성북구     14
송파구     17
양천구     14
영등포구    14
용산구     14
은평구     17
종로구      3
중구      10
중구구      6
중랑구      2
dtype: int64

연도별 총 강수량 평균:
일자
2018    15.596266
2019    12.434273
2020    17.412732
2021    10.936649
2022    21.553090
2023    16.387130
Name: